# Enhance reproducability of results

Since all ML algorithms are inherently stochastic, running them multiple times give different solutions. However, for debugging and model performance comparison, it is important to have a degree of reproducability in the results.

In [ ]:
#%env PYTHONHASHSEED=0 
#%env CUDA_VISIBLE_DEVICES=""

In [ ]:
import numpy as np
import tensorflow as tf
import random
import os
tf.logging.set_verbosity(tf.logging.ERROR)

np.random.seed(1)
random.seed(1)
tf.set_random_seed(1)

config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=config)
K.set_session(sess)

In [ ]:
def resetRNG(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.set_random_seed(seed_value)

# Module IMPORTS

In [ ]:
#from keras.layers import Input, Dense, Flatten, Reshape, concatenate, Conv2D, Conv1D
#from keras.layers import SeparableConv1D, Lambda, Conv2DTranspose, LeakyReLU
from keras.layers import *
from keras.models import Model, load_model
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K 
#from keras import initializers
import keras

from IPython.display import SVG, display
from keras.utils.vis_utils import plot_model, model_to_dot

# Import DATABASE

Import the .npz Database containing 5SS mechanisms and cplr paths.

In [ ]:
def import_DB_offline():
    current_dir=os.getcwd()
    db_path=os.path.join(current_dir,'..','1. DB Preprocessing','norm_database5SS.npz')
    db = np.load(db_path)
    cplr=db['cplrData']
    mech=db['mechData']
    
    return cplr, mech

In [ ]:
def import_DB_colab():
    from google.colab import drive
    
    drive.mount('/content/drive')
    %cd /content/drive/My\ Drive/Colab\ Notebooks
    db = np.load('norm_database5SS.npz')
    cplr=db['cplrData']
    mech=db['mechData']
    
    return cplr, mech

# Autoencoder TRAINING

In [ ]:
# Train same model architecture multiple times since the training process is stochastic

def AE_analysis(AE_model, iterations, x_train, x_test, filename='model'):
    # AE_model: Keras Autoencoder model
    # iterations: Times the autoencoder is trained from scratch. Only best model saved.
    
    val_loss_list=[]
    best_val_loss=np.inf;
    for i in range(iterations):
        print('Model no. '+str(i+1))
        AE, E = AE_model()
        val_loss, train_data=train_AE(AE, E, x_train, x_test)
        val_loss_list.append(val_loss)
        if best_val_loss>val_loss:
            best_AE=AE
            best_E=E
            best_train_data=train_data
            best_val_loss=val_loss
    
    # PRINT MEAN, VARIANCE and BEST MODEL METRICS
    print('**************************************************')
    print('Validation Loss Stastics:')
    print('Values: '+str(val_loss_list))
    print('Mean: '+str(np.around(np.mean(val_loss_list), decimals=4)))
    print('Standard Deviation: '+str(np.around(np.std(val_loss_list), decimals=4)))
    print('**************************************************')
    
    # SAVE BEST MODEL
    AEfilename='AE_'+filename+'_val_loss_'+str(best_val_loss)+'.h5'
    AEfilepath=os.path.join('models',AEfilename)
    best_AE.save(AEfilepath)
    
    return best_AE, best_E, best_train_data

def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def train_AE(autoencoder, encoder, x_train, x_test, epochs=1000):
    # TRAINING PHASE
    autoencoder.compile(optimizer='adam', loss='mean_absolute_error', 
                        metrics=['accuracy', 'mse', rmse])
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min',
                                  restore_best_weights=True)
    autoencoder_train=autoencoder.fit(x_train, x_train, epochs=epochs, batch_size=128, 
                                      shuffle=False, validation_data=(x_test, x_test), 
                                      verbose=0, callbacks=[earlyStopping])
    
    # TRAINING HISTORY
    val_loss = autoencoder_train.history['val_loss']
    f_val_loss=np.round_(val_loss[-1],decimals=4)
    
    return f_val_loss, autoencoder_train

# Autoencoder VISUALIZATION

In [ ]:
def visualize_AE(AE, train_data, x_test):
    # VISUALIZING MODEL ARCHITECTURE
    AE.summary()
    display(SVG(model_to_dot(AE, show_shapes=True, show_layer_names=True).create(prog='dot', format='svg')))
    
    # VISUALIZING TRAINING CURVES
    visualize_train_curves(train_data)
    
    # VISUALIZE RECONSTRUCTED CURVES
    reconst_curve = AE.predict(x_test)
    visualize_reconstructed_CplrCurves(x_test, reconst_curve, 10)

    
def visualize_train_curves(train_data):
    loss = train_data.history['loss']
    val_loss = train_data.history['val_loss']
    
    f_loss=np.round_(loss[-1],decimals=4)
    f_val_loss=np.round_(val_loss[-1],decimals=4)
    
    print('Training loss: '+str(f_loss))
    print('Validation loss: '+str(f_val_loss))
    
    epochsRange = range(len(loss))
    plt.figure(figsize=(14, 7))
    plt.plot(epochsRange, loss, 'r', label='Training loss')
    plt.plot(epochsRange, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

    
def visualize_reconstructed_CplrCurves(x_test, reconst_curve, n=10, b_spline=False):
    fig= plt.figure(figsize=(3*4, 3*n))
    fig.suptitle('Isometric (3D), Top (X-Y Plane), Front (X-Z Plane) and Right (Y-Z Plane) View', fontsize=14, fontweight='bold')
    for i in range(n):
        # Isometric View
        ax = plt.subplot(n, 4, 4*i+1, projection='3d')
        if b_spline:
            reconst_curve[i]=b_spline_interpolation(reconst_curve[i], 100)
        plotPath3D(reconst_curve[i], ax, 1,'r' )
        plotPath3D(x_test[i], ax, 1)
        
        #x-y, x-z, y-z plane view
        xy_recon=reconst_curve[i,:,0:2]
        xy_orig=x_test[i,:,0:2]
        xz_recon=np.concatenate(([reconst_curve[i,:,0]], [reconst_curve[i,:,2]]),axis=0).T
        xz_orig=np.concatenate(([x_test[i,:,0]], [x_test[i,:,2]]),axis=0).T
        yz_recon=reconst_curve[i,:,1:3]
        yz_orig=x_test[i,:,1:3]
        
        # Top View
        ax = plt.subplot(n, 4, 4*i+2)
        plotPath(xy_recon, ax, 1,'r' )
        plotPath(xy_orig, ax, 1)
        
        # Front View
        ax = plt.subplot(n, 4, 4*i+3)
        plotPath(xz_recon, ax, 1,'r' )
        plotPath(xz_orig, ax, 1)
        
        # Right View
        ax = plt.subplot(n, 4, 4*i+4)
        plotPath(yz_recon, ax, 1,'r' )
        plotPath(yz_orig, ax, 1)
        
    fig.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

In [ ]:
# Plotting Functions

%matplotlib inline
#%matplotlib notebook
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from scipy import interpolate


def plotPath3D(Pts, ax, limit, color = 'gray'):
    xline=Pts[:,0]
    yline=Pts[:,1]
    zline=Pts[:,2]
    ax.plot3D(xline, yline, zline, color)
    ax.auto_scale_xyz([0, limit], [0, limit], [0, limit])
    
def plotPath(Pts, ax, limit, color = 'gray'):
    xline=Pts[:,0]
    yline=Pts[:,1]
    ax.plot(xline, yline, color)
    ax.set(xlim=(0, limit), ylim=(0, limit))

def plotXYZ(center, RotMat, ax):
    C=np.vstack((center,center,center))
    R=RotMat
    r=(1,0,0)
    g=(0,1,0)
    b=(0,0,1)
    ax.quiver(C[:,0], C[:,1], C[:,2], R[:,0], R[:,1], R[:,2],color=(r,g,b,r,r,g,g,b,b))


def b_spline_interpolation(inp_pts, out_n):
    # Fit cubic B-spline to the points
    xp=inp_pts[i,:,0]
    yp=inp_pts[i,:,1]
    zp=inp_pts[i,:,2]
        
    # Check for duplicate points as interpolation routine errors out
    okay = np.where(np.abs(np.diff(xp)) + np.abs(np.diff(yp)) + np.abs(np.diff(zp)) > 0)
    xp = np.r_[xp[okay], xp[-1]]
    yp = np.r_[yp[okay], yp[-1]]
    zp = np.r_[zp[okay], zp[-1]]
    
    tck, u =interpolate.splprep([xp,yp,zp],s=1)
    num_pts=out_n
    u_fine = np.linspace(0,1,num_pts)
    x_f, y_f, z_f = interpolate.splev(u_fine, tck)
    Path_f = np.vstack(([x_100],[y_100],[z_100])).T
    
    return Path_f

## Visualize variation of coupler curves with change in z-space parameters

# Autoencoder Architectures: 

In [2]:
######################################################################
# Sigmoid Z_30 hidden_2 [300,100,100,30,100,100,300]
def aeSig():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(100, activation='sigmoid')(x)
    x=Dense(100, activation='sigmoid')(x)
    encoded=Dense(30, activation='sigmoid')(x)

    # Decoding
    x=Dense(100, activation='sigmoid')(encoded)
    x=Dense(100, activation='sigmoid')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# RELU Z_30 hidden_2 [300,100,100,30,100,100,300]
def aeRELU():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(100, activation='relu')(x)
    x=Dense(100, activation='relu')(x)
    encoded=Dense(30, activation='relu')(x)

    # Decoding
    x=Dense(100, activation='relu')(encoded)
    x=Dense(100, activation='relu')(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# LRELU Z_30 hidden_2 [300,100,100,30,100,100,300]
def aeLRELU():
    inp_curve=Input(shape=(100,3,))

    # Encoding
    x=Flatten()(inp_curve)
    x=Dense(100)(x)
    x=LeakyReLU(alpha=0.01)(x)
    x=Dense(100)(x)
    x=LeakyReLU(alpha=0.01)(x)
    x=Dense(30)(x)
    encoded=LeakyReLU(alpha=0.01)(x)

    # Decoding
    x=Dense(100)(encoded)
    x=LeakyReLU(alpha=0.01)(x)
    x=Dense(100)(x)
    x=LeakyReLU(alpha=0.01)(x)
    x=Dense(300)(x)
    decoded=Reshape((100, 3))(x)

    autoencoder = Model(inp_curve, decoded)
    encoder = Model(inp_curve, encoded)
    
    return autoencoder, encoder

######################################################################
# Z_2 hidden_2
